In [ ]:
import fasttext
from sklearn.decomposition import TruncatedSVD

In [ ]:
import gensim
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd

FILE = '../data/canada_data_select.csv'
df_data = pd.read_csv(FILE)
df_data.drop_duplicates(['ad_id', 'locations', 'phone_id', 'title', 'body'], inplace=True)
df_data.reset_index(inplace=True)

all_words = []


stop_words = set(stopwords.words('english'))
content = []
for index, row in df_data.iterrows():
    c = ''
    # if type(row['title']) == type(' '):
    # 	c = c + row['title']
    if type(row['body']) == type(' '):
        c = c + row['body']
    c = re.sub(r'\d+', '', c)
    c = re.sub(r'[^\x00-\x7F]+','', c)
    cleanr = re.compile('<.*?>')
    c = re.sub(cleanr, '', c)
    c = re.sub(r"[^a-zA-Z0-9.?]+", ' ', c)
    content.append(c.lower())
    if index%10000 == 0:
        print (index)

df_data = pd.read_csv('../data/ht_unique_unique_ads_cosine.csv')
for index, row in df_data.iterrows():
    c = ''
    # if type(row['title']) == type(' '):
    # 	c = c + row['title']
    if type(row['body']) == type(' '):
        c = c + row['body']
    c = re.sub(r'\d+', '', c)
    c = re.sub(r'[^\x00-\x7F]+','', c)
    cleanr = re.compile('<.*?>')
    c = re.sub(cleanr, '', c)
    c = re.sub(r"[^a-zA-Z0-9.?]+", ' ', c)
    content.append(c.lower())
    
print ("Writing to file....")
with open('../data/canada/only_text_train1.txt', 'a') as f:
    f.writelines(content)

print (len(content)) 

# def filter_words(word):
#     word = str(word)
#     if any(not x.isalpha() for x in word):
#         return True

#     elif len(word) < 3:
#         return True

#     return False

# words_total = []
# for index, sent in enumerate(content):
#     sent_words = []
#     try:
#         sentences = nltk.sent_tokenize(sent)
#     except Exception as e:
#         print (e)
#         continue
    
#     for ind, sen in enumerate(sentences):
#         words = list(nltk.word_tokenize(sen))

#         for w in words:
#             w = re.sub('\W+','', w)
#             if filter_words(w):
#                 continue
#             sent_words.append(w.lower())

#     if sent_words:
#         with open('../data/canada_select_only_text1.txt', 'a') as f:
#             f.write(' '.join(sent_words))
#             f.write('\n')

# print (len(all_words))


In [ ]:
fasttext_model = fasttext.train_unsupervised('../data/canada/only_text_train1.txt', model='skipgram')
word_vectors = fasttext_model

In [ ]:
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

def filter_words(word):
    word = str(word)
    if any(not x.isalpha() for x in word):
        return True

    elif len(word) < 3:
        return True

    return False

df_data = pd.read_csv('../data/ht_unique_unique_ads_cosine.csv')
content = []

stop_words = set(stopwords.words('english'))
for index, row in df_data.iterrows():
    c = ''
    # if type(row['title']) == type(' '):
    # 	c = c + row['title']
    if type(row['body']) == type(' '):
        c = c + row['body']
    # c = row['title'] + ' ' + row['body']
    c = re.sub(r'\d+', '', c)
    if type(row['Name']) == type(''):
        name = row['Name'].split(';')
        for n in name:
            name_regex = re.compile(re.escape(n), re.IGNORECASE)
            c = name_regex.sub('', c)
        c = re.sub(r'[^\x00-\x7F]+',' ', c)
        cleanr = re.compile('<.*?>')
        c = re.sub(cleanr, '', c)
    content.append(c)
df_data['content_p'] = content

filtered_content = []
all_words = []

for index, sent in enumerate(content):
    sent_words = []
    
    try:
        sentences = nltk.sent_tokenize(sent)
    except Exception as e:
        print (e)
        continue
    
    for ind, sen in enumerate(sentences):
        words = list(nltk.word_tokenize(sen))

        for w in words:
            w = re.sub('\W+','', w)
            if filter_words(w):

                continue
            sent_words.append(w.lower())

    if not sent_words:
        df_data.drop(index, inplace=True)
    else:
        all_words.append(sent_words)
        filtered_content.append(' '.join(sent_words))
    
vectorizer = TfidfVectorizer(lowercase=True, ngram_range=(1,1), norm='l2', 
     stop_words=stop_words, min_df=2, max_df=0.8)

bigram_matrix = vectorizer.fit_transform(filtered_content)
features_col = vectorizer.get_feature_names()
bigram_dense_matrix = np.asarray(bigram_matrix.todense())

In [ ]:
vectors = np.zeros((len(all_words), 100), dtype=float)
total_not_found = 0.0
not_found_count = 0.0

for ind, word_tokens in enumerate(all_words):
    not_found_count = 0
    for word in word_tokens:
        weight = 0.0
        try:
            word_index = features_col.index(word)
            weight = bigram_dense_matrix[ind][word_index]
        except:
#             print (word)
            not_found_count += 1
        if word in word_vectors and weight != 0.0:
            vectors[ind] += (weight * 10 * word_vectors[word])

    total_not_found += not_found_count
    print ("Not Found : {}".format(not_found_count/len(word_tokens)))


In [ ]:
true_labels = df_data['label'].values.tolist()

binary_true_labels = [0] * len(true_labels)
for ind, label in enumerate(true_labels):
    if label >= 4:
        binary_true_labels[ind] = 1

In [ ]:
df_data['binary_label'] = binary_true_labels
# df_unique = df_orig.copy()

In [ ]:
colors = ['green', 'red', 'blue']
color_array = []
for ind, l in enumerate(binary_true_labels):
    color_array.append(colors[int(l)])

In [ ]:
import umap
X_embedded = umap.UMAP().fit_transform(vectors)

In [ ]:
# ######################### HBDSCAN ############################
import hdbscan
clusterer = hdbscan.HDBSCAN(min_cluster_size=3, algorithm='best', alpha=1.0)
clusterer.fit(vectors)
#     print (clusterer.labels_)
labels = clusterer.labels_
labels_clustering = labels
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
unique_labels = set(labels)
probs = clusterer.probabilities_
#     colors = [plt.cm.Spectral(each)
#               for each in np.linspace(0, 1, len(set(true_labels))]
print ("Number of labels : " + str(len(list(set(clusterer.labels_)))))
#     palette = sns.color_palette()
#     cluster_colors = [sns.desaturate(palette[col], sat)
#                       if col >= 0 else (0.5, 0.5, 0.5) for col, sat in
#                       zip(clusterer.labels_, clusterer.probabilities_)]

In [ ]:

binary_true_labels = np.asarray(binary_true_labels)
# noise_labels = np.where(labels==-1)
no_noise_labels = np.where(labels!=-1)
tr_labels = np.where(binary_true_labels==1)
print (binary_true_labels.shape)
print (no_noise_labels[0].shape)
print (tr_labels)
# lst3 = [value for value in tr_labels if value in noise_labels] 
common_tp = np.intersect1d(no_noise_labels[0], tr_labels[0])
print (common_tp.shape)
total_p = len(common_tp)/len((tr_labels[0]))
print (total_p)

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

from pylab import rcParams
rcParams['figure.figsize'] = 20,10
plt.scatter(X_embedded.T[0], X_embedded.T[1], c=color_array, alpha=0.2)
# plt.title("Clustering accuracy={}, fmeasure_synth={}, number_of_labels={}".format(clustering_acc, fmeasure, 
#                                                                                   len(unique_labels)))
plt.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False,
right='off', left='off', labelleft='off')
# plt.text(-5, 10, 'FastText', fontsize=24)
plt.savefig('../results/embedding_fasttext.png')
plt.show()

In [ ]:
def get_all_subgraphs(mat_data, cl_ind):
    filtered_sub = mat_data[cl_ind,:][:]
    bigrams_count = np.count_nonzero(filtered_sub, axis=0)
#     print ("Bigrams count matrix shape : {}".format(bigrams_count.shape))
    zero_count_index = np.where(bigrams_count==0)[1]
#     print (zero_count_index)
    one_count_index = list(np.where(bigrams_count==1)[1])
#     print ("One count index : {}".format(one_count_index))
#     print ("Max zero count index : {} ".format(max(zero_count_index)))
    core_bigrams_index = list(np.where(bigrams_count>1)[1])
#     print ("Core Bigrams index : {}".format(len(core_bigrams_index)))
    outer_bigrams_index = list(one_count_index + core_bigrams_index)
    
    bigram_induced_graph = mat_data[:][:,core_bigrams_index]
#     print ("Bigram Induced Graph shape : {}".format(bigram_induced_graph.shape))
    ads_count = np.count_nonzero(bigram_induced_graph, axis=1)
    ads_in_shell_index = np.where(ads_count>1)[0]
    
    not_core_ads = list(set(list(ads_in_shell_index)) - set(cl_ind))
    not_core_bigrams = list(set(list(one_count_index)) - set(core_bigrams_index))
    mat_copy = np.asarray(mat_data)
    for i in not_core_ads:
        for j in not_core_bigrams:
            mat_copy[i][j] = 0
    shell_subgraph = mat_copy[ads_in_shell_index,:][:,outer_bigrams_index]
    mat_copy[not_core_ads, :] = 0
    outer_subgraph = mat_copy[ads_in_shell_index,:][:,outer_bigrams_index]
    mat_copy[:,one_count_index] = 0
    core_subgraph = mat_copy[ads_in_shell_index,:][:,outer_bigrams_index]
#     print ("Shell Subgraph shape : {}". format(shell_subgraph.shape))
#     print ("Outer Subgraph shape : {}". format(outer_subgraph.shape))
#     print ("Core Subgraph shape : {}". format(core_subgraph.shape))
#     print (zero_count_index.shape)
#     print (zero_count_index[1])
#     outer_subgraph = np.delete(filtered_sub, zero_count_index, axis=1)
#     core_subgraph = outer_subgraph.copy()
#     core_subgraph[:,one_count_index] = 0
#     core_subgraph = np.delete(filtered_sub, list(set(list(zero_count_index) + list(one_count_index))), axis=1)
    shell_subgraph = np.asarray(shell_subgraph)
    outer_subgraph = np.asarray(outer_subgraph)
    core_subgraph = np.asarray(core_subgraph)
#     print ("subgraph sizes: {}, {}, {}".format(core_subgraph.shape, outer_subgraph.shape, shell_subgraph.shape))
    
    return shell_subgraph, outer_subgraph, core_subgraph

In [ ]:
bigram_matrix = bigram_matrix.todense()

In [ ]:
import math

def calculate_unweighted_density(core_mat):
    edge_weight = np.count_nonzero(core_mat)
    ads_count = np.count_nonzero(core_mat, axis=0)
    bigrams_count = np.count_nonzero(core_mat, axis=1)
    ads_core_num = len(np.where(ads_count>0)[0])
    bigrams_core_num = len(np.where(bigrams_count>0)[0])
#     print (ads_core_num)
#     print (bigrams_core_num)
    return edge_weight/(ads_core_num * bigrams_core_num + 1)

def calculate_weighted_density(core_mat):
    edge_weight = np.sum(core_mat)
    ads_count = np.count_nonzero(core_mat, axis=0)
    bigrams_count = np.count_nonzero(core_mat, axis=1)
    ads_core_num = len(np.where(ads_count>0)[0])
    bigrams_core_num = len(np.where(bigrams_count>0)[0])

    return edge_weight/(ads_core_num * bigrams_core_num + 1)

def calculate_unweighted_fraudar_score(core_mat):
    edge_weight = np.count_nonzero(core_mat)
    ads_count = np.count_nonzero(core_mat, axis=0)
    bigrams_count = np.count_nonzero(core_mat, axis=1)
    ads_core_num = len(np.where(ads_count>0)[0])
    bigrams_core_num = len(np.where(bigrams_count>0)[0])
    
    return edge_weight/(ads_core_num + bigrams_core_num + 1)

def calculate_weighted_fraudar_score(core_mat):
    edge_weight = np.sum(core_mat)
    ads_count = np.count_nonzero(core_mat, axis=0)
    bigrams_count = np.count_nonzero(core_mat, axis=1)
    ads_core_num = len(np.where(ads_count>0)[0])
    bigrams_core_num = len(np.where(bigrams_count>0)[0])
    
    return edge_weight/(ads_core_num + bigrams_core_num + 1)

def calculate_unweighted_edge_per_score(core_mat, outer_mat):
    core_edges = np.count_nonzero(core_mat)
    outer_edges = np.count_nonzero(outer_mat)
    bigrams_count = np.count_nonzero(core_mat, axis=1)
    bigrams_core_num = len(np.where(bigrams_count>0)[0])
    
    return ((core_edges + 1)/(outer_edges +1))*(math.log(bigrams_core_num+1))

def calculate_weighted_edge_per_score(core_mat, outer_mat):
    core_edges = np.sum(core_mat)
    outer_edges = np.sum(outer_mat)
    
    bigrams_count = np.count_nonzero(core_mat, axis=1)
    bigrams_core_num = len(np.where(bigrams_count>0)[0])
    
    return ((core_edges + 1)/(outer_edges +1))*(math.log(bigrams_core_num+1))


def calculate_custom_score(core_mat, outer_mat):
    bigram_degrees = np.count_nonzero(np.asarray(core_mat), axis=0)
    bigrams_count = np.count_nonzero(core_mat, axis=1)
    bigrams_core_num = len(np.where(bigrams_count>0)[0])
    outer_edges = np.count_nonzero(outer_mat)
    bigram_degrees = bigram_degrees/bigrams_core_num
    
    ads_count = np.count_nonzero(core_mat, axis=0)
    ads_core_num = len(np.where(ads_count>0)[0])
    
    return (np.sum(bigram_degrees)/(outer_edges+1))*(math.log(bigrams_core_num+1))*(math.log(ads_core_num+1))
# #     print (mat.shape)
#     edges_nonzero = np.count_nonzero(mat, axis=0)
#     unique, counts = np.unique(edges_nonzero, return_counts=True)
#     degree_counts = dict(zip(unique, counts))
#     numerator = 0.0
#     denominator = 0.0
#     half = max(mat.shape[0]/2, 2)
#     for k, v in degree_counts.items():
#         if k == 0:
#             continue
#         elif k <= half:
#             denominator += k*v
#         else:
#             denominator += k*v
#             numerator += k*v
#     if denominator == 0.0:
#         return 0.0
#     else:
#         return numerator/denominator

# def calculate_weighted_edge_per_score(mat):
#     return 0.0

#Should be shell_mat instead of outer_mat, change once you figure out how to get shell subgraph.
def calculate_unweighted_modularity_score(core_mat, outer_mat, total_edges):
#     total_edges = math.log(total_edges)
    ad_degrees = np.count_nonzero(np.asarray(outer_mat), axis=1)
#     print (ad_degrees)
    bigram_degrees = np.count_nonzero(np.asarray(outer_mat), axis=0)
#     print (bigram_degrees)
    ads_count = np.count_nonzero(core_mat, axis=1)
    bigrams_count = np.count_nonzero(core_mat, axis=0)
    ads_core_index = np.where(ads_count>0)[0]
    bigrams_core_index = np.where(bigrams_count>0)[0]
#     ads_list_index = [x for x in range(core_mat.shape[0])]
#     nonzero_edges = np.transpose(np.nonzero(core_mat))
    summation = 0.0
    for i in ads_core_index:
        for j in bigrams_core_index:
            if core_mat[i][j] == 0:
                adj = 0
            else:
                adj = 1
            if adj == 1:
                summation += (adj - (ad_degrees[i] * bigram_degrees[j])/total_edges)

#     nonzero_edges = np.transpose(np.nonzero(core_mat))
#     summation = 0.0
#     for k in range(nonzero_edges.shape[0]):
#         ad_index = nonzero_edges[k][0]
#         big_index = nonzero_edges[k][1]
# #         print (ad_index)
# #         print (big_index)
# #         if core_mat[ad_index][big_index] != 0:
# #             adj = 1
# #         else:
# #             adj = 0
        
#         summation += (1 - (ad_degrees[ad_index] * bigram_degrees[big_index])/(2*total_edges))
    
    return (summation/total_edges)*(math.log(len(ads_core_index)+1))

def calculate_weighted_modularity_score(core_mat, outer_mat, total_edges):
    ad_degrees = np.sum(np.asarray(outer_mat), axis=1)
    bigram_degrees = np.sum(np.asarray(outer_mat), axis=0)
    
    ads_count = np.count_nonzero(core_mat, axis=1)
    bigrams_count = np.count_nonzero(core_mat, axis=0)
    ads_core_index = np.where(ads_count>0)[0]
    bigrams_core_index = np.where(bigrams_count>0)[0]

    

#     ads_list_index = [x for x in range(core_mat.shape[0])]
#     nonzero_edges = np.transpose(np.nonzero(core_mat))
    summation = 0.0
    for i in ads_core_index:
        for j in bigrams_core_index:
            if core_mat[i][j] != 0:
                summation += (core_mat[i][j] - (ad_degrees[i] * bigram_degrees[j])/total_edges)
#     summation = 0.0
#     for k in range(nonzero_edges.shape[0]):
#         ad_index = nonzero_edges[k][0]
#         big_index = nonzero_edges[k][1]
#         summation += (outer_mat[ad_index][big_index] - (ad_degrees[ad_index] * bigram_degrees[big_index])/(2*total_edges))
    
    return (summation/total_edges)*(math.log(len(ads_core_index)+1))

def calculate_pairwise_modularity(mat):
    mat = np.asarray(mat.todense())
    sim_scores = np.zeros((mat.shape[0], mat.shape[0]))
    for i in range(mat.shape[0]):
        for j in range(i+1, mat.shape[0]):
#             print ("i : {}, j : {}".format(i,j))
            if i == j:
                continue
#             print (len(mat[i]))
            sim_scores[i][j] = calculate_modularity_score(np.vstack((mat[i], mat[j])))
    
    return sim_scores

In [ ]:
import gc
from sklearn.metrics.pairwise import cosine_similarity

def get_all_metrics(bigram_matrix, unique_labels, labels, df_data):
    eigen_ratios = []
    weighted_cluster_density = []
    unweighted_cluster_density = []
    unweighted_fraudar_scores = []
    weighted_fraudar_scores = []
    unweighted_outer_edge_perc_scores = []
    weighted_outer_edge_perc_scores = []
    unweighted_shell_edge_perc_scores = []
    weighted_shell_edge_perc_scores = []
    weighted_outer_modularity_scores = []
    unweighted_outer_modularity_scores = []
    weighted_shell_modularity_scores = []
    unweighted_shell_modularity_scores = []
    pairwise_similarity = []
    custom_score = []
    avg_label_scores = []
    max_label_scores = []
    sum_label_scores = []
    avg_binary_scores = []
    max_binary_scores = []
    sum_binary_scores = []
    clusters = []
    cluster_counts = []
    
    total_edges_unweighted = np.count_nonzero(bigram_matrix)
    total_edges_weighted = np.sum(bigram_matrix)
    for l in unique_labels:
#         s = bigram_matrix.sum(axis=1)
        if l== -1:
            weighted_cluster_density.append(0)
            unweighted_cluster_density.append(0)
            weighted_fraudar_scores.append(0)
            unweighted_fraudar_scores.append(0)
            weighted_outer_edge_perc_scores.append(0)
            unweighted_outer_edge_perc_scores.append(0)
            weighted_shell_edge_perc_scores.append(0)
            unweighted_shell_edge_perc_scores.append(0)
            unweighted_outer_modularity_scores.append(0)
            weighted_outer_modularity_scores.append(0)
            unweighted_shell_modularity_scores.append(0)
            weighted_shell_modularity_scores.append(0)
            pairwise_similarity.append(0)
            custom_score.append(0)
            cluster_counts.append(math.log(len(cluster_idx), 10)) 
            eigen_ratios.append(0)
            clusters.append(l)
            
            max_label_scores.append(0)
            avg_label_scores.append(0)
            sum_label_scores.append(0)

            max_binary_scores.append(0)
            avg_binary_scores.append(0)
            sum_binary_scores.append(0)
            continue
#         print (s.shape)
#         print ("bigram matrix sum : {}".format(bigram_matrix.sum()))
#         print ("Zero elems: {}".format(len(np.argwhere(s==0))))
        cluster_idx = np.argwhere(labels == l).reshape(-1)
#         print (l, len(cluster_idx))
        
        print (cluster_idx)
        shell_subgraph, outer_subgraph, core_subgraph = get_all_subgraphs(bigram_matrix.copy(), cluster_idx)
#         print (l, len(cluster_idx), core_subgraph.sum(), outer_subgraph.sum(), shell_subgraph.sum())
        

        df_filt = df_data[df_data['cluster_label']== l]
        if len(df_filt) == 0 or core_subgraph.shape[0] == 0 or shell_subgraph.shape[0] == 0 or outer_subgraph.shape[0] == 0:
            weighted_cluster_density.append(0)
            unweighted_cluster_density.append(0)
            weighted_fraudar_scores.append(0)
            unweighted_fraudar_scores.append(0)
            weighted_outer_edge_perc_scores.append(0)
            unweighted_outer_edge_perc_scores.append(0)
            weighted_shell_edge_perc_scores.append(0)
            unweighted_shell_edge_perc_scores.append(0)
            unweighted_outer_modularity_scores.append(0)
            weighted_outer_modularity_scores.append(0)
            unweighted_shell_modularity_scores.append(0)
            weighted_shell_modularity_scores.append(0)
            pairwise_similarity.append(0)
            custom_score.append(0)
            cluster_counts.append(math.log(len(cluster_idx), 10)) 
            eigen_ratios.append(0)
            clusters.append(l)
            
            max_label_scores.append(0)
            avg_label_scores.append(0)
            sum_label_scores.append(0)

            max_binary_scores.append(0)
            avg_binary_scores.append(0)
            sum_binary_scores.append(0)
            continue
#         elif core_subgraph.shape[0] == 0 or shell_subgraph.shape[0] == 0 or outer_subgraph.shape[0] == 0:
#             continue
        print (l, df_filt.shape)
        max_label_scores.append(max(df_filt['label']))
        avg_label_scores.append(sum(df_filt['label'])/len(df_filt['label']))
        sum_label_scores.append(sum(df_filt['label']))

        max_binary_scores.append(max(df_filt['binary_label']))
        avg_binary_scores.append(sum(df_filt['binary_label'])/len(df_filt['binary_label']))
        sum_binary_scores.append(sum(df_filt['binary_label']))
        
        local_content = list(df_filt['content_p'])
        count_vectorizer = TfidfVectorizer(ngram_range=(2,2), use_idf=False)
        count_data = count_vectorizer.fit_transform(local_content)

        svd = TruncatedSVD(n_components=2)
        local_vecs = svd.fit_transform(count_data)
        w = svd.singular_values_
        if len(w) > 1:
            eigen_rat = w[1]/w[0]
        else:
            eigen_rat = 1
        eigen_ratios.append(eigen_rat)
        
        print (outer_subgraph.shape)
        print (core_subgraph.shape)
        pairwise_sim_mat = cosine_similarity(outer_subgraph, dense_output=True)
        pairwise_sim_mat = np.tril(pairwise_sim_mat, -1)
#         print (sum(pairwise_sim_mat).shape)
        print (pairwise_sim_mat.sum())
        an_score = calculate_weighted_edge_per_score(core_subgraph, outer_subgraph)
        names = [x.lower() if type(x) == type('') else None for x in df_filt['Name'].unique()]
        if math.nan in names:
            names.remove(math.nan)
        if None in names:
            names.remove(None)
        names = list(set(names))
        c_score = an_score * max(0, len(names)-1)
        print ("Scores : {}, {}, {}".format(an_score, len(names), c_score))
        weighted_cluster_density.append(calculate_weighted_density(core_subgraph))
        unweighted_cluster_density.append(calculate_unweighted_density(core_subgraph))
        weighted_fraudar_scores.append(calculate_weighted_fraudar_score(core_subgraph))
        unweighted_fraudar_scores.append(calculate_unweighted_fraudar_score(core_subgraph))
        weighted_outer_edge_perc_scores.append(calculate_weighted_edge_per_score(core_subgraph, outer_subgraph))
        unweighted_outer_edge_perc_scores.append(calculate_unweighted_edge_per_score(core_subgraph, outer_subgraph))
        weighted_shell_edge_perc_scores.append(calculate_weighted_edge_per_score(core_subgraph, shell_subgraph))
        unweighted_shell_edge_perc_scores.append(calculate_unweighted_edge_per_score(core_subgraph, shell_subgraph))
        unweighted_outer_modularity_scores.append(calculate_unweighted_modularity_score(core_subgraph, outer_subgraph, total_edges_unweighted))
        weighted_outer_modularity_scores.append(calculate_weighted_modularity_score(core_subgraph, outer_subgraph, total_edges_weighted))
        unweighted_shell_modularity_scores.append(calculate_unweighted_modularity_score(core_subgraph, shell_subgraph, total_edges_unweighted))
        weighted_shell_modularity_scores.append(calculate_weighted_modularity_score(core_subgraph, shell_subgraph, total_edges_weighted))
        custom_score.append(c_score)
        pairwise_similarity.append(pairwise_sim_mat.sum()/len(cluster_idx))
        cluster_counts.append(math.log(len(cluster_idx), 10)) 
        clusters.append(l)
        
        count_data = []
        local_content = []
        shell_subgraph = []
        core_subgraph = []
        outer_subgraph = []
        if l % 50 == 0:
            print (l)
            gc.collect()
#     original_labels = labels.copy()

    metrics = {}
    metrics['weighted_cluster_density'] = weighted_cluster_density
    metrics['unweighted_cluster_density'] = unweighted_cluster_density
    metrics['weighted_fraudar_scores'] = weighted_fraudar_scores
    metrics['unweighted_fraudar_scores'] = unweighted_fraudar_scores
    metrics['weighted_outer_edge_perc_scores'] = weighted_outer_edge_perc_scores
    metrics['unweighted_outer_edge_perc_scores'] = unweighted_outer_edge_perc_scores
    metrics['weighted_shell_edge_perc_scores'] = weighted_shell_edge_perc_scores
    metrics['unweighted_shell_edge_perc_scores'] = unweighted_shell_edge_perc_scores
    metrics['unweighted_outer_modularity_scores'] = unweighted_outer_modularity_scores
    metrics['weighted_outer_modularity_scores'] = weighted_outer_modularity_scores
    metrics['unweighted_shell_modularity_scores'] = unweighted_shell_modularity_scores
    metrics['weighted_shell_modularity_scores'] = weighted_shell_modularity_scores
    metrics['pairwise_similarity'] = pairwise_similarity
    metrics['custom_score'] = custom_score
    metrics['avg_label_scores'] = avg_label_scores
    metrics['sum_label_scores'] = sum_label_scores
    metrics['max_label_scores'] = max_label_scores
    metrics['avg_binary_scores'] = avg_binary_scores
    metrics['max_binary_scores'] = max_binary_scores
    metrics['sum_binary_scores'] = sum_binary_scores
    metrics['eigen_ratios'] = eigen_ratios
    metrics['clusters'] = clusters
    metrics['labels'] = labels.copy()
    metrics['cluster_counts'] = cluster_counts
    
    return metrics


In [ ]:
pre_merging_metrics = get_all_metrics(bigram_matrix, unique_labels, labels, df_data)

In [ ]:
suspicious_scores = pre_merging_metrics['custom_score']
text_similarity = pre_merging_metrics['pairwise_similarity']
clusters = pre_merging_metrics['clusters']
avg_label_scores = pre_merging_metrics['avg_label_scores']
avg_binary_scores = pre_merging_metrics['avg_binary_scores']
w_density = pre_merging_metrics['weighted_cluster_density']
uw_density = pre_merging_metrics['unweighted_cluster_density']
w_fraudar = pre_merging_metrics['weighted_fraudar_scores']
uw_fraudar = pre_merging_metrics['unweighted_fraudar_scores']
w_outer_edge = pre_merging_metrics['weighted_outer_edge_perc_scores']
uw_outer_edge = pre_merging_metrics['unweighted_outer_edge_perc_scores']
w_shell_edge = pre_merging_metrics['weighted_shell_edge_perc_scores']
uw_shell_edge = pre_merging_metrics['unweighted_shell_edge_perc_scores']
w_outer_mod = pre_merging_metrics['weighted_outer_modularity_scores']
uw_outer_mod = pre_merging_metrics['unweighted_outer_modularity_scores']
w_shell_mod = pre_merging_metrics['weighted_shell_modularity_scores']
uw_shell_mod = pre_merging_metrics['unweighted_shell_modularity_scores']

In [ ]:
import seaborn as sns


ax = sns.regplot(x=avg_label_scores, y=suspicious_scores, x_ci=68, truncate=False, lowess=True, scatter_kws = {'alpha': 0.2}, line_kws = {'color': 'red'})
ax.set(xlabel='Average Label Score', ylabel='anomaly score')
plt.show()

ax = sns.regplot(x=avg_label_scores, y=w_outer_edge, x_ci=68, truncate=False, lowess=True, scatter_kws = {'alpha': 0.2}, line_kws = {'color': 'red'})
ax.set(xlabel='Average Label Score', ylabel='Weighted outer edge percentage')
plt.show()

# import seaborn as sns
ax = sns.regplot(x=text_similarity, y=w_outer_edge, ci=68, truncate=False, lowess=True, scatter_kws = {'color': 'b', 'alpha': 0.2}, line_kws = {'color': 'red'})
ax.set(xlabel='Text Similarity', ylabel='Weighted outer edge percentage')
plt.show()


ax = sns.regplot(x=avg_label_scores, y=text_similarity, ci=68, truncate=False, lowess=True, scatter_kws = {'color': 'b', 'alpha': 0.2}, line_kws = {'color': 'red'})
ax.set(xlabel='Average Label Score', ylabel='Text Similarity')
plt.show()
# import seaborn as sns
# ax = sns.regplot(x=avg_label_scores, y=w_outer_edge, ci=68, truncate=False, lowess=True, scatter_kws = {'color': 'b', 'alpha': 0.2}, line_kws = {'color': 'red'})
# ax.set(xlabel='Average Label Score', ylabel='Weighted outer edge percentage')
# plt.show()

# import seaborn as sns
# ax = sns.regplot(x=avg_label_scores, y=w_outer_edge, ci=68, truncate=False, lowess=True, scatter_kws = {'color': 'b', 'alpha': 0.2}, line_kws = {'color': 'red'})
# ax.set(xlabel='Average Label Score', ylabel='Weighted outer edge percentage')
# plt.show()

# import seaborn as sns
# ax = sns.regplot(x=avg_label_scores, y=w_outer_edge, ci=68, truncate=False, lowess=True, scatter_kws = {'color': 'b', 'alpha': 0.2}, line_kws = {'color': 'red'})
# ax.set(xlabel='Average Label Score', ylabel='Weighted outer edge percentage')
# plt.show()

# import seaborn as sns
# ax = sns.regplot(x=avg_label_scores, y=w_outer_edge, ci=68, truncate=False, lowess=True, scatter_kws = {'color': 'b', 'alpha': 0.2}, line_kws = {'color': 'red'})
# ax.set(xlabel='Average Label Score', ylabel='Weighted outer edge percentage')
# plt.show()


# plt.scatter(avg_label_scores, w_outer_edge, alpha=0.2)
# plt.title('Avg label score vs weighted outer edge percentage')
# plt.xlabel('avg label score')
# plt.ylabel('weighted outer edge percentage')
# # w = np.linalg.lstsq(avg_label_scores, w_outer_edge)[0]
# # yh = np.dot(avg_label_scores,w)
# # plt.plot(avg_label_scores, yh, 'r-')
# plt.show()

# plt.scatter(avg_label_scores, w_shell_edge, alpha=0.2)
# plt.title('Avg label score vs weighted shell edge percentage')
# plt.xlabel('avg label score')
# plt.ylabel('weighted shell edge percentage')
# plt.show()

# plt.scatter(text_similarity, w_outer_edge, alpha=0.2)
# plt.title('Avg text similarity vs weighted outer edge percentage')
# plt.xlabel('avg text similarity')
# plt.ylabel('weighted outer edge percentage')

# # plt.scatter(avg_label_scores, w_shell_edge, alpha=0.2)
# # plt.title('Avg label score vs weighted shell edge percentage')
# # plt.xlabel('avg label score')
# # plt.ylabel('weighted shell edge percentage')
# plt.show()

In [ ]:
def filter_by_name(clusters, df_data):
    filtered_clusters = []
    for cl in clusters:
        df_fil = df_data[df_data['cluster_label']==cl]
        names = [x.lower() if type(x) == type('') else None for x in df_fil['Name'].unique()]
        if math.nan in names:
            names.remove(math.nan)
        if None in names:
            names.remove(None)
        names = list(set(names))
        if len(names)>1:
#             print (names)
            filtered_clusters.append(cl)
#             filtered_cluster_metric.append(anomaly_cluster_metric[ind])
#         else:
#             filtered_clusters.append(c)
#             filtered_cluster_metric.append(anomaly_cluster_metric[ind])
    return filtered_clusters

def filter_by_threshold(clusters, metric, threshold=0.0):
    filtered_clusters = []
    index_list = [i for i, e in enumerate(metric) if e > threshold]
    filtered_clusters = [clusters[i] for i in index_list] 
    
    return filtered_clusters

In [ ]:
filtered_clusters = filter_by_name(clusters, df_data.copy())
print (filtered_clusters)
filtered_clusters = set(filtered_clusters)
index_list = [i for i, e in enumerate(clusters) if e in filtered_clusters]
filtered_avg_label_scores = [avg_label_scores[i] for i in index_list] 
filtered_text_similarity = [text_similarity[i] for i in index_list] 
filtered_suspicious_scores = [suspicious_scores[i] for i in index_list] 
filtered_avg_binary_scores = [avg_binary_scores[i] for i in index_list] 
filtered_w_density = [w_density[i] for i in index_list] 
filtered_uw_density = [uw_density[i] for i in index_list] 
filtered_w_fraudar = [w_fraudar[i] for i in index_list] 
filtered_uw_fraudar = [uw_fraudar[i] for i in index_list] 
filtered_w_outer_edge = [w_outer_edge[i] for i in index_list] 
filtered_uw_outer_edge = [uw_outer_edge[i] for i in index_list] 
filtered_w_shell_edge = [w_shell_edge[i] for i in index_list] 
filtered_uw_shell_edge = [uw_shell_edge[i] for i in index_list] 
filtered_w_outer_mod = [w_outer_mod[i] for i in index_list] 
filtered_uw_outer_mod = [uw_outer_mod[i] for i in index_list] 
filtered_w_shell_mod = [w_shell_mod[i] for i in index_list] 
filtered_uw_shell_mod = [uw_shell_mod[i] for i in index_list] 

In [ ]:
from scipy.stats import spearmanr
import plotly.express as px

filtered_suspicious_scores, filtered_clusters, filtered_avg_label_scores, filtered_text_similarity, \
filtered_w_density, filtered_uw_density, filtered_w_fraudar,  filtered_uw_fraudar, filtered_w_outer_edge, \
filtered_uw_outer_edge, filtered_w_shell_edge, filtered_uw_shell_edge,  filtered_w_outer_mod, \
filtered_uw_outer_mod, filtered_w_shell_mod, filtered_uw_shell_mod = \
zip(*sorted(zip(filtered_suspicious_scores, filtered_clusters, filtered_avg_label_scores, filtered_text_similarity, 
                filtered_w_density, filtered_uw_density, filtered_w_fraudar,  filtered_uw_fraudar, 
                filtered_w_outer_edge, filtered_uw_outer_edge, filtered_w_shell_edge, filtered_uw_shell_edge,  
                filtered_w_outer_mod, filtered_uw_outer_mod, filtered_w_shell_mod, filtered_uw_shell_mod)))
filtered_suspicious_scores = list(reversed(filtered_suspicious_scores))
filtered_clusters = list(reversed(filtered_clusters))
filtered_avg_label_scores = list(reversed(filtered_avg_label_scores))
filtered_text_similarity = list(reversed(filtered_text_similarity))
filtered_avg_binary_scores = list(reversed(filtered_avg_binary_scores))
filtered_w_density = list(reversed(filtered_w_density))
filtered_uw_density = list(reversed(filtered_uw_density))
filtered_w_fraudar = list(reversed(filtered_w_fraudar))  
filtered_uw_fraudar = list(reversed(filtered_uw_fraudar))
filtered_w_outer_edge = list(reversed(filtered_w_outer_edge)) 
filtered_uw_outer_edge = list(reversed(filtered_w_density)) 
filtered_w_shell_edge = list(reversed(filtered_w_shell_edge))
filtered_uw_shell_edge = list(reversed(filtered_uw_shell_edge))  
filtered_w_outer_mod = list(reversed(filtered_w_outer_mod)) 
filtered_uw_outer_mod = list(reversed(filtered_uw_outer_mod)) 
filtered_w_shell_mod = list(reversed(filtered_w_shell_mod)) 
filtered_uw_shell_mod = list(reversed(filtered_uw_shell_mod))
top_k = [10, 50, 100, 150, 200, 250, 300, 350, 400]
rcParams['figure.figsize'] = 4,4
for k in top_k:
    print ('==================================k = {}========================================'.format(k) )
    plt.hist(filtered_avg_label_scores[:k], bins=7)
    plt.show()
    
    plt.hist(filtered_avg_binary_scores[:k], bins=3)
    plt.show()
    
    plt.plot(filtered_text_similarity[:k], marker='o', linestyle='--')
    plt.xlabel(' Clusters')
    plt.ylabel('avg text similarity')
    plt.show()
    
    print ("Average of average label scores : {}".format(sum(filtered_avg_label_scores[:k])/k))
    print ("Average text similarity : {}".format(sum(filtered_text_similarity[:k])/k))
    print ('\n')
    eig_ts_pscore = spearmanr(filtered_avg_label_scores[:k], filtered_text_similarity[:k])[0]
    eig_w_den_pscore = spearmanr(filtered_avg_label_scores[:k], filtered_w_density[:k])[0]
    eig_uw_den_pscore = spearmanr(filtered_avg_label_scores[:k], filtered_uw_density[:k])[0]
    eig_w_fraud_pscore = spearmanr(filtered_avg_label_scores[:k], filtered_w_fraudar[:k])[0]
    eig_uw_fraud_pscore = spearmanr(filtered_avg_label_scores[:k], filtered_uw_fraudar[:k])[0]
    eig_w_edge_out_pscore = spearmanr(filtered_avg_label_scores[:k], filtered_w_outer_edge[:k])[0]
    eig_uw_edge_out_pscore = spearmanr(filtered_avg_label_scores[:k], filtered_uw_outer_edge[:k])[0]
    eig_w_edge_shell_pscore = spearmanr(filtered_avg_label_scores[:k], filtered_w_shell_edge[:k])[0]
    eig_uw_edge_shell_pscore = spearmanr(filtered_avg_label_scores[:k], filtered_uw_shell_edge[:k])[0]
    eig_w_mod_out_pscore = spearmanr(filtered_avg_label_scores[:k], filtered_w_outer_mod[:k])[0]
    eig_uw_mod_out_pscore = spearmanr(filtered_avg_label_scores[:k], filtered_uw_outer_mod[:k])[0]
    eig_w_mod_shell_pscore = spearmanr(filtered_avg_label_scores[:k], filtered_w_shell_mod[:k])[0]
    eig_uw_mod_shell_pscore = spearmanr(filtered_avg_label_scores[:k], filtered_uw_shell_mod[:k])[0]
    
    print ("Spearman Correlation | Average Label Score |          Text Similarity          | {0:.2f} ".format(eig_ts_pscore))
    print ("Spearman Correlation | Average Label Score |     Weighted Cluster Density      | {0:.2f} ".format(eig_w_den_pscore))
    print ("Spearman Correlation | Average Label Score |    UnWeighted Cluster Density     | {0:.2f} ".format(eig_uw_den_pscore))
    print ("Spearman Correlation | Average Label Score |      Weighted Fraudar Score       | {0:.2f} ".format(eig_w_fraud_pscore))
    print ("Spearman Correlation | Average Label Score |     UnWeighted Fraudar Score      | {0:.2f} ".format(eig_uw_fraud_pscore))
    print ("Spearman Correlation | Average Label Score |     Weighted Outer Edge Perc      | {0:.2f} ".format(eig_w_edge_out_pscore))
    print ("Spearman Correlation | Average Label Score |    UnWeighted Outer Edge Perc     | {0:.2f} ".format(eig_uw_edge_out_pscore))
    print ("Spearman Correlation | Average Label Score |     Weighted Shell Edge Perc      | {0:.2f} ".format(eig_w_edge_out_pscore))
    print ("Spearman Correlation | Average Label Score |    UnWeighted Shell Edge Perc     | {0:.2f} ".format(eig_uw_edge_out_pscore))
    print ("Spearman Correlation | Average Label Score |  Weighted Outer Modularity Score  | {0:.2f} ".format(eig_w_mod_out_pscore))
    print ("Spearman Correlation | Average Label Score | UnWeighted Outer Modularity Score | {0:.2f} ".format(eig_uw_mod_out_pscore))
    print ("Spearman Correlation | Average Label Score |  Weighted Shell Modularity Score  | {0:.2f} ".format(eig_w_mod_shell_pscore))
    print ("Spearman Correlation | Average Label Score | UnWeighted Shell Modularity Score | {0:.2f} ".format(eig_uw_mod_shell_pscore))
    
    eig_w_den_pscore = spearmanr(filtered_avg_binary_scores[:k], filtered_w_density[:k])[0]
    eig_uw_den_pscore = spearmanr(filtered_avg_binary_scores[:k], filtered_uw_density[:k])[0]
    eig_w_fraud_pscore = spearmanr(filtered_avg_binary_scores[:k], filtered_w_fraudar[:k])[0]
    eig_uw_fraud_pscore = spearmanr(filtered_avg_binary_scores[:k], filtered_uw_fraudar[:k])[0]
    eig_w_edge_out_pscore = spearmanr(filtered_avg_binary_scores[:k], filtered_w_outer_edge[:k])[0]
    eig_uw_edge_out_pscore = spearmanr(filtered_avg_binary_scores[:k], filtered_uw_outer_edge[:k])[0]
    eig_w_edge_shell_pscore = spearmanr(filtered_avg_binary_scores[:k], filtered_w_shell_edge[:k])[0]
    eig_uw_edge_shell_pscore = spearmanr(filtered_avg_binary_scores[:k], filtered_uw_shell_edge[:k])[0]
    eig_w_mod_out_pscore = spearmanr(filtered_avg_binary_scores[:k], filtered_w_outer_mod[:k])[0]
    eig_uw_mod_out_pscore = spearmanr(filtered_avg_binary_scores[:k], filtered_uw_outer_mod[:k])[0]
    eig_w_mod_shell_pscore = spearmanr(filtered_avg_binary_scores[:k], filtered_w_shell_mod[:k])[0]
    eig_uw_mod_shell_pscore = spearmanr(filtered_avg_binary_scores[:k], filtered_uw_shell_mod[:k])[0]
    
    print ('\n')
    print ("Spearman Correlation | Average Binary Score |     Weighted Cluster Density      | {0:.2f} ".format(eig_w_den_pscore))
    print ("Spearman Correlation | Average Binary Score |    UnWeighted Cluster Density     | {0:.2f} ".format(eig_uw_den_pscore))
    print ("Spearman Correlation | Average Binary Score |      Weighted Fraudar Score       | {0:.2f} ".format(eig_w_fraud_pscore))
    print ("Spearman Correlation | Average Binary Score |     UnWeighted Fraudar Score      | {0:.2f} ".format(eig_uw_fraud_pscore))
    print ("Spearman Correlation | Average Binary Score |     Weighted Outer Edge Perc      | {0:.2f} ".format(eig_w_edge_out_pscore))
    print ("Spearman Correlation | Average Binary Score |    UnWeighted Outer Edge Perc     | {0:.2f} ".format(eig_uw_edge_out_pscore))
    print ("Spearman Correlation | Average Binary Score |     Weighted Shell Edge Perc      | {0:.2f} ".format(eig_w_edge_out_pscore))
    print ("Spearman Correlation | Average Binary Score |    UnWeighted Shell Edge Perc     | {0:.2f} ".format(eig_uw_edge_out_pscore))
    print ("Spearman Correlation | Average Binary Score |  Weighted Outer Modularity Score  | {0:.2f} ".format(eig_w_mod_out_pscore))
    print ("Spearman Correlation | Average Binary Score | UnWeighted Outer Modularity Score | {0:.2f} ".format(eig_uw_mod_out_pscore))
    print ("Spearman Correlation | Average Binary Score |  Weighted Shell Modularity Score  | {0:.2f} ".format(eig_w_mod_shell_pscore))
    print ("Spearman Correlation | Average Binary Score | UnWeighted Shell Modularity Score | {0:.2f} ".format(eig_uw_mod_shell_pscore))
print (len(clusters))

In [ ]:
rcParams['figure.figsize'] = 10,10

ax = sns.regplot(x=filtered_avg_label_scores, y=filtered_suspicious_scores, x_ci=68, truncate=False, lowess=True, scatter_kws = {'alpha': 0.2}, line_kws = {'color': 'red'})
ax.set(xlabel='Average Label Score', ylabel='anomaly score')
plt.title('After filtering out clusters with only one individual')
plt.show()

ax = sns.regplot(x=filtered_avg_label_scores, y=filtered_w_outer_edge, x_ci=68, truncate=False, lowess=True, scatter_kws = {'alpha': 0.2}, line_kws = {'color': 'red'})
ax.set(xlabel='Average Label Score', ylabel='Weighted outer edge percentage')
plt.title('After filtering out clusters with only one individual')
plt.show()

# import seaborn as sns
ax = sns.regplot(x=filtered_text_similarity, y=filtered_w_outer_edge, ci=68, truncate=False, lowess=True, scatter_kws = {'color': 'b', 'alpha': 0.2}, line_kws = {'color': 'red'})
ax.set(xlabel='Text Similarity', ylabel='Weighted outer edge percentage')
plt.title('After filtering out clusters with only one individual')
plt.show()


ax = sns.regplot(x=filtered_avg_label_scores, y=filtered_text_similarity, ci=68, truncate=False, lowess=True, scatter_kws = {'color': 'b', 'alpha': 0.2}, line_kws = {'color': 'red'})
ax.set(xlabel='Average Label Score', ylabel='Text Similarity')
plt.title('After filtering out clusters with only one individual')
plt.show()

In [ ]:
suspicious_scores = pre_merging_metrics['weighted_outer_edge_perc_scores']
text_similarity = pre_merging_metrics['pairwise_similarity']
clusters = pre_merging_metrics['clusters']
avg_label_scores = pre_merging_metrics['avg_label_scores']
avg_binary_scores = pre_merging_metrics['avg_binary_scores']
w_density = pre_merging_metrics['weighted_cluster_density']
uw_density = pre_merging_metrics['unweighted_cluster_density']
w_fraudar = pre_merging_metrics['weighted_fraudar_scores']
uw_fraudar = pre_merging_metrics['unweighted_fraudar_scores']
w_outer_edge = pre_merging_metrics['weighted_outer_edge_perc_scores']
uw_outer_edge = pre_merging_metrics['unweighted_outer_edge_perc_scores']
w_shell_edge = pre_merging_metrics['weighted_shell_edge_perc_scores']
uw_shell_edge = pre_merging_metrics['unweighted_shell_edge_perc_scores']
w_outer_mod = pre_merging_metrics['weighted_outer_modularity_scores']
uw_outer_mod = pre_merging_metrics['unweighted_outer_modularity_scores']
w_shell_mod = pre_merging_metrics['weighted_shell_modularity_scores']
uw_shell_mod = pre_merging_metrics['unweighted_shell_modularity_scores']
cluster_counts = pre_merging_metrics['cluster_counts']

In [ ]:
suspicious_scores, clusters, avg_label_scores, text_similarity, \
w_density, uw_density, w_fraudar,  uw_fraudar, w_outer_edge, \
uw_outer_edge, w_shell_edge, uw_shell_edge,  w_outer_mod, \
uw_outer_mod, w_shell_mod, uw_shell_mod, cluster_counts = \
zip(*sorted(zip(suspicious_scores, clusters, avg_label_scores, text_similarity, 
                w_density, uw_density, w_fraudar,  uw_fraudar, w_outer_edge, 
                uw_outer_edge, w_shell_edge, uw_shell_edge,  w_outer_mod, 
                uw_outer_mod, w_shell_mod, uw_shell_mod, cluster_counts)))
suspicious_scores = list(reversed(suspicious_scores))
clusters = list(reversed(clusters))
avg_label_scores = list(reversed(avg_label_scores))
text_similarity = list(reversed(text_similarity))
# filtered_avg_binary_scores = list(reversed(filtered_avg_binary_scores))
w_density = list(reversed(w_density))
uw_density = list(reversed(uw_density))
w_fraudar = list(reversed(w_fraudar))  
uw_fraudar = list(reversed(uw_fraudar))
w_outer_edge = list(reversed(w_outer_edge)) 
uw_outer_edge = list(reversed(uw_outer_edge)) 
w_shell_edge = list(reversed(w_shell_edge))
uw_shell_edge = list(reversed(uw_shell_edge))  
w_outer_mod = list(reversed(w_outer_mod)) 
uw_outer_mod = list(reversed(uw_outer_mod)) 
w_shell_mod = list(reversed(w_shell_mod)) 
uw_shell_mod = list(reversed(uw_shell_mod))
cluster_counts = list(reversed(cluster_counts))

In [ ]:
content = list(df_data['content_p'])
vectorizer = TfidfVectorizer(lowercase=True, ngram_range=(2, 3), norm='l2', 
    smooth_idf=True, stop_words=stop_words, min_df=2, max_df=0.8)
# where_are_nans = np.isnan(content)
# content[where_are_nans] = 0
bigram_matrix = vectorizer.fit_transform(content)

print (bigram_matrix.shape)

In [ ]:
df_data['cluster_label'] = labels

In [ ]:
new_index = range(len(df_data))
df_data = df_data.reindex(new_index)
# print (df_da.index)

In [ ]:
k=50
sort_ind = []
for ind, cl in enumerate(clusters[:k]):
    if cl==-1 or cl==-2:
        continue
    else:
        df_f = df_data[df_data['cluster_label'] == cl]
        print (df_f.shape)
#         if df_f.shape[0] < 20:
#             continue
        sort_ind += list(df_f.index)
print (sort_ind)

In [ ]:

sim_matrix = cosine_similarity(bigram_matrix)
print (sim_matrix.shape)
# fig = plt.figure(figsize=(5, 5))
# ax = fig.add_subplot(111)
# ax.tick_params(
#     axis='both',          # changes apply to the x-axis
#     which='both',      # both major and minor ticks are affected
#     bottom=False,      # ticks along the bottom edge are off
#     top=False,         # ticks along the top edge are off
#     labelbottom=False,
# right='off', left='off', labelleft='off')
cmap = sns.cubehelix_palette(50, hue=0.05, rot=0, light=0.9, dark=0, as_cmap=True)
ax = sns.heatmap(sim_matrix[sort_ind, :][:,sort_ind], cmap=cmap, xticklabels=False, yticklabels=False, cbar=False)
# ax.set(xlabel='cosine similarity', ylabel='cosine_similarity')
plt.savefig('../results/sim_matrix.png')

In [ ]:
noise_index = clusters.index(-1)
del suspicious_scores[noise_index]
del text_similarity[noise_index]
del clusters[noise_index]
del avg_label_scores[noise_index]
del avg_binary_scores[noise_index]
del w_density[noise_index]
del uw_density[noise_index]
del w_fraudar[noise_index]
del uw_fraudar[noise_index]
del w_outer_edge[noise_index]
del uw_outer_edge[noise_index]
del w_shell_edge[noise_index]
del uw_shell_edge[noise_index]
del w_outer_mod[noise_index]
del uw_outer_mod[noise_index]
del w_shell_mod[noise_index]
del uw_shell_mod[noise_index]
del cluster_counts[noise_index]

In [ ]:
cluster_counts = [10**x for x in cluster_counts]

In [ ]:
k = 50

In [ ]:
total_ads_in_clusters = sum(cluster_counts)
avg_of_avg_label_scores = sum(avg_label_scores[:k])/k
avg_of_anomaly_scores = sum(suspicious_scores[:k])/k
avg_text_similarity = sum(text_similarity[:k])/k
avg_cluster_size = sum(cluster_counts[:k])/k

print ("Total ads in clusters: {}, Average of label scores per cluster: {}, \
average of anomaly scores per cluster: {}, Average text sim: {}, avg cluster size: {}".format(
total_ads_in_clusters, avg_of_avg_label_scores, avg_of_anomaly_scores, avg_text_similarity, avg_cluster_size))

In [ ]:
%%timeit
b =1 

In [ ]:
b